#Task 1

In [12]:
import random

Values = [(7, 5), (2, 4), (1, 7), (9, 2)]
Population = [[random.randint(0, 1) for i in range(4)] for j in range(20)]

def Fitness(chromosome):
    curWeight, curVal = 0, 0
    for gene, (value, weight) in zip(chromosome, Values):
        if gene:
            curWeight += weight
            curVal += value

    return curVal if curVal <= 15 else 0

def crossover(childOne, childTwo):
    ind = random.randint(0, len(Values) - 1)
    return childOne[:ind] + childTwo[ind:], childTwo[:ind] + childOne[ind:]

def mutate(chromosome):
    ind = random.randint(0, 3)
    chromosome[ind] = 1 if chromosome[ind] == 0 else 0
    return chromosome

def LinearRankingSelection(Population):
    sortedPop = sorted(Population, key=Fitness)
    n = len(sortedPop)

    # (2 - s)/n + 2i(s - 1)/(n(n - 1)), where s is selection pressure
    s = 1.7
    probs = [(2 - s)/n + 2*i*(s - 1)/(n*(n - 1)) for i in range(n)]

    r = random.random()
    cumulative = 0
    for ind, p in zip(sortedPop, probs):
        cumulative += p
        if r < cumulative:
            return ind
    return sortedPop[-1]


def BinaryKnapsack(Population):
    for i in range(50):
        sortedPopulation = sorted(Population, key = Fitness, reverse = True)[:int(0.4*len(Population))]

        while len(sortedPopulation) < 20:
            parentOne, parentTwo = LinearRankingSelection(sortedPopulation), LinearRankingSelection(sortedPopulation)
            childOne, childTwo = crossover(parentOne, parentTwo)

            if random.random() < 0.1:
                childOne = mutate(childOne)
            if random.random() < 0.1:
                childTwo = mutate(childTwo)

            sortedPopulation.extend([childOne, childTwo])

        Population = sortedPopulation
    bestChromosome = max(sortedPopulation, key = Fitness)
    bestVal = Fitness(bestChromosome)

    return bestChromosome, bestVal

In [13]:
Chromosome, Value = BinaryKnapsack(Population)

print(f"Best Chromosome: {Chromosome}\nValue: {Value}")

Best Chromosome: [0, 1, 1, 1]
Value: 12


#Task 2 (Without Pruning)

In [14]:
import math

def MiniMax(score, turn):
    #   True for Player, False for AI
    if score >= 14:
        return -1 if turn else 1

    if turn:
        bestMove = -math.inf
        for move in [1, 2, 3]:
            currentScore = MiniMax(score + move, not turn)
            bestMove = max(bestMove, currentScore)
        return bestMove
    else:
        bestMove = math.inf
        for move in [1, 2, 3]:
            currentScore = MiniMax(score + move, not turn)
            bestMove = min(bestMove, currentScore)
        return bestMove


def bestMoveForAI(score):
    bestMove = 0
    bestScore = math.inf

    for move in [1, 2, 3]:
        if move + score <= 14:
            result = MiniMax(score + move, True)
            if bestScore > result:
                bestScore = result
                bestMove = move
    return bestMove

def Game(score = 14):
    turn = True

    currentScore = 0

    while currentScore < 14:
        if turn:
            move = int(input(f"Current Score: {currentScore}\tEnter a number [1, 2, 3]: "))
            if move not in [1, 2, 3]:
                print("Invalid move. Try again.")
                continue
            currentScore += move
            turn = not turn
        else:
            move = bestMoveForAI(currentScore)
            currentScore += move
            turn = not turn
            print(f"Current Score: {currentScore}\tAI chose to add {move}")

    if not turn:
        print("You have defeated my AI! Congratulations!")
    else:
        print("Better Luck Next Time!")

Game()

Current Score: 0	Enter a number [1, 2, 3]: 3
Current Score: 6	AI chose to add 3
Current Score: 6	Enter a number [1, 2, 3]: 3
Current Score: 10	AI chose to add 1
Current Score: 10	Enter a number [1, 2, 3]: 1
Current Score: 14	AI chose to add 3
Better Luck Next Time!


#Task 3 (Alpha Beta Pruning)

In [15]:
import math

def MiniMaxAlphaBeta(score, turn, alpha, beta):
    if score >= 14:
        return -1 if turn else 1

    if turn:
        bestMove = -math.inf
        for move in [1, 2, 3]:
            currentScore = MiniMaxAlphaBeta(score + move, not turn, alpha, beta)
            bestMove = max(bestMove, currentScore)
            alpha = max(alpha, bestMove)
            if alpha >= beta:
                break
        return bestMove
    else:
        bestMove = math.inf
        for move in [1, 2, 3]:
            currentScore = MiniMaxAlphaBeta(score + move, not turn, alpha, beta)
            bestMove = min(bestMove, currentScore)
            beta = min(beta, bestMove)
            if beta <= alpha:
                break
        return bestMove


def bestMoveForAIAlphaBeta(score):
    bestMove = 0
    bestScore = math.inf

    alpha = -math.inf
    beta = math.inf

    for move in [1, 2, 3]:
        if move + score <= 14:
            result = MiniMaxAlphaBeta(score + move, True, alpha, beta)
            if bestScore > result:
                bestScore = result
                bestMove = move
    return bestMove

def GameAlphaBeta():
    turn = True
    score = 14
    currentScore = 0

    while currentScore < score:
        if turn:
            move = int(input(f"Current Score: {currentScore}.\tEnter your Move [1, 2, 3]"))
            if move not in [1, 2, 3]:
                print("Invalid Move! Try Again.")
                continue
            currentScore += move
            turn = not turn
        else:
            move = bestMoveForAIAlphaBeta(currentScore)
            currentScore += move
            turn = not turn
            print(f"Current Score: {currentScore}\tAI chose to add {move}")

    if not turn:
        print("You have defeated my AI! Congratulations!")
    else:
        print("Better Luck Next Time!")

GameAlphaBeta()

Current Score: 0.	Enter your Move [1, 2, 3]3
Current Score: 6	AI chose to add 3
Current Score: 6.	Enter your Move [1, 2, 3]3
Current Score: 10	AI chose to add 1
Current Score: 10.	Enter your Move [1, 2, 3]1
Current Score: 14	AI chose to add 3
Better Luck Next Time!
